In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [15]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [16]:
%%capture pwd
!pwd

In [17]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-can-the-northern-irish-healthcare-system-be-fixed', 300, 500)

# Fig 1

In [18]:
df = pd.read_csv("raw/hs-niwts-tables-outpatients-q3-21-22.csv").dropna(subset=['HSC Trust'])
# df = pd.read_csv("raw/hs-niwts-tables-icats-q3-21-22.csv").dropna(subset=['HSC Trust'])
for c in df.columns[-20:]:
    df[c]=df[c].astype(str).replace('--',np.nan).str.replace(',','').astype(float)
df=df.groupby(['Quarter Ending']).sum()
df['<9 weeks']=df['Total Waiting']-df['>52 weeks']-df['>18-52 weeks']-\
    -df['> 15-18 weeks']-df['> 9 - 12 weeks']-df['> 12-15 weeks']
df['>9-52 weeks']=df['Total Waiting']-df['<9 weeks']-df['>52 weeks']
df=df.stack().reset_index().rename(columns={0:'value','level_1':'duration'})
df['order']=df['duration'].replace('>52 weeks',2).replace('>9-52 weeks',1).replace('<9 weeks',0)

In [19]:
f = "fig1_waiitng_times"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Quarter Ending,duration,value,order
0,30-Jun-16,0 - 6 weeks,59989.0,0 - 6 weeks
1,30-Jun-16,> 6 - 9 weeks,19437.0,> 6 - 9 weeks
2,30-Jun-16,> 9 - 13 weeks,0.0,> 9 - 13 weeks
3,30-Jun-16,> 9 - 12 weeks,16879.0,> 9 - 12 weeks
4,30-Jun-16,> 12-15 weeks,11209.0,> 12-15 weeks


In [20]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "Quarter Ending:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    tooltip=['Quarter Ending:T','value:Q']
)
bars1 = base.mark_bar(color=colors["eco-gray"]).encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            gridOpacity=0.4,
            title="Total waiting",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
).transform_filter('datum.duration=="Total Waiting"')
bars2 = base.mark_bar(color=colors["eco-mid-blue"]).encode(
    y=alt.Y("value:Q"),
    color=alt.Color('duration:N',scale=alt.Scale(
        range=[colors['eco-light-blue'],colors['eco-blue'],colors['eco-mid-blue']]),legend=None),
    order='order:Q'
).transform_filter('datum["Quarter Ending"]>datetime(2015,7)')\
.transform_filter(alt.FieldOneOfPredicate(field='duration',oneOf=['<9 weeks','>9-52 weeks','>52 weeks']))
text=bars2.mark_text(align='left',dx=10).encode(text='duration:N')\
.transform_filter('datum["Quarter Ending"]>datetime(2021,11)')
layer1 = (
    ((bars1+bars2+text).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)